## GPU
在PyTorch中，每个数组都有⼀个设备（device），我们通常将其称为环境（context）。默认情况下，所有
变量和相关的计算都分配给CPU。有时环境可能是GPU。当我们跨多个服务器部署作业时，事情会变得更加
棘⼿。通过智能地将数组分配给环境，我们可以最⼤限度地减少在设备之间传输数据的时间。例如，当在带
有GPU的服务器上训练神经⽹络时，我们通常希望模型的参数在GPU上。


In [1]:
!nvidia-smi

Tue Mar 21 11:16:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:26:00.0  On |                  N/A |
| 29%   37C    P0    N/A /  30W |   1228MiB /  2048MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 计算设备
我们可以指定⽤于存储和计算的设备，如CPU和GPU。默认情况下，张量是在内存中创建的，然后使⽤CPU计
算它。  

在PyTorch中，CPU和GPU可以⽤torch.device('cpu') 和torch.device('cuda')表⽰。应该注意的是，cpu设
备意味着所有物理CPU和内存，这意味着PyTorch的计算将尝试使⽤所有CPU核⼼。然⽽，gpu设备只代表⼀
个卡和相应的显存。如果有多个GPU，我们使⽤torch.device(f'cuda:{i}') 来表⽰第i块GPU（i从0开始）。
另外，cuda:0和cuda是等价的。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')


(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
# 查询可⽤gpu的数量
torch.cuda.device_count()

1

In [4]:
# 定义了两个⽅便的函数，这两个函数允许我们在不存在所需所有GPU的情况下运⾏代码
def try_gpu(i=0): #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus(): #@save
    """返回所有可⽤的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

### 张量与GPU
可以查询张量所在的设备。默认情况下，张量是在CPU上创建的。


In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，⽆论何时我们要对多个项进⾏操作，它们都必须在同⼀个设备上。例如，如果我们对两个张
量求和，我们需要确保两个张量都位于同⼀个设备上，否则框架将不知道在哪⾥存储结果，甚⾄不知道在哪
⾥执⾏计算。

### 存储在GPU上
有⼏种⽅法可以在GPU上存储张量。例如，我们可以在创建张量时指定存储设备。接下来，我们在第⼀个gpu上
创建张量变量X。在GPU上创建的张量只消耗这个GPU的显存。我们可以使⽤nvidia-smi命令查看显存使⽤情
况。⼀般来说，我们需要确保不创建超过GPU显存限制的数据。


In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [8]:
# 假设我们⾄少有两个GPU，下⾯的代码将在第⼆个GPU上创建⼀个随机张量。（这里我们只有一个GPU，所以改一下）
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.6174, 0.9236, 0.1390],
        [0.0169, 0.4551, 0.1881]], device='cuda:0')

#### 复制
如果我们要计算X + Y，我们需要决定在哪⾥执⾏这个操作。例如，如 图5.6.1所⽰，我们可以将X传输到第⼆
个GPU并在那⾥执⾏操作。不要简单地X加上Y，因为这会导致异常，运⾏时引擎不知道该怎么做：它在同⼀
设备上找不到数据会导致失败。由于Y位于第⼆个GPU上，所以我们需要将X移到那⾥，然后才能执⾏相加运
算。


In [10]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [11]:
# 现在数据在同⼀个GPU上（Z和Y都在），我们可以将它们相加。
Y + Z

tensor([[1.6174, 1.9236, 1.1390],
        [1.0169, 1.4551, 1.1881]], device='cuda:0')

### 神经网络与GPU
类似地，神经⽹络模型可以指定设备。下⾯的代码将模型参数放在GPU上。


In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())
net(X)


tensor([[-0.5338],
        [-0.5338]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [14]:
# 们确认模型参数存储在同⼀个GPU上
net[0].weight.data.device

device(type='cuda', index=0)

### 小结
* 可以指定⽤于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使⽤CPU进⾏计算。
* 深度学习框架要求计算的所有输⼊数据都在同⼀设备上，⽆论是CPU还是GPU。
* 不经意地移动数据可能会显著降低性能。⼀个典型的错误如下：计算GPU上每个⼩批量的损失，并在命令⾏中将其报告给⽤⼾（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从⽽使所有GPU阻塞。最好是为GPU内部的⽇志分配内存，并且只移动较⼤的⽇志。